In [1]:
# Import libraries:
import xarray as xr
import numpy as np
import pandas as pd
import os
from six.moves import urllib
from glob import glob
import matplotlib.pyplot as plt
import matplotlib  as mpl 
import pickle
import pyart
from datetime import datetime
import math
from pandas.core.common import flatten
%matplotlib inline
# %matplotlib widget
import tobac
from tobac.merge_split import merge_split_MEST

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

/Users/kelcy/opt/anaconda3/envs/TTU_KNB/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/Users/kelcy/opt/anaconda3/envs/TTU_KNB/lib/python3.8/site-packages/iris/fileformats/_ff.py:819: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def _parse_binary_stream(file_like, dtype=np.float, count=-1):
/Users/kelcy/opt/anaconda3/envs/TTU_KNB/lib/python3.8/site-packages/pyke/knowledge_engine.py:28: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/Users/kelcy/opt/anaconda3/envs/TTU_KNB/lib/python3.8/site-packages/sklearn/utils/multiclass.py:14: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  f

In [4]:
# Import tobac itself:
import tobac
from tobac.merge_split import merge_split_MEST
#Disable a couple of warnings:
# import warnings
# warnings.filterwarnings('ignore', category=UserWarning, append=True)
# warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
# warnings.filterwarnings('ignore', category=FutureWarning, append=True)
# warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [5]:

path = '/Users/kelcy/PYTHON/TRACER-PAWS-NEXRAD-LMA/scripts/Gridding/'

data = xr.open_mfdataset(path+'*grid.nc', engine = 'netcdf4',compat='override',coords = 'minimal')

data['time'].encoding['units']="seconds since 2000-01-01 00:00:00"
bad_rhv = data["cross_correlation_ratio"] < 0.9
bad_refl = data["reflectivity"] < 10
bad=bad_rhv & bad_refl
maxrefl = data["reflectivity"].where(~bad, np.nan).max(axis=1)

In [7]:
ts = pd.to_datetime(data['time'][50].values)
date = ts.strftime('%Y%m%d')
print(date)

20230621


In [8]:
track_threshold =  30 

In [40]:
#Set up directory to save output and plots:
savedir='tobac_Save_'+date
if not os.path.exists(savedir):
    os.makedirs(savedir)
plot_dir=savedir+"/tobac_Plot"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

#Feature detection:
#Feature detection is perfomed based on surface precipitation field and a range of thresholds

In [41]:
# Dictionary containing keyword options (could also be directly given to the function)
parameters_features={}
parameters_features['position_threshold']='weighted_diff'
parameters_features['sigma_threshold']=1.0#0.5 #5.0 #is the default
parameters_features['threshold']= track_threshold #15
#parameters_features['min_num']=0
#parameters_features['min_distance']=5 #0 #15
#parameters_features['n_erosion_threshold']=0
#parameters_features['n_min_threshold']=0

In [42]:
#This is an alternative method for finding date/time in km/min
datetimes = data['time']
timedeltas = [datetimes[i-1]-datetimes[i] for i in range(1, len(datetimes))]
average_timedelta = sum(timedeltas) / len(timedeltas)
dt = np.abs((np.mean(np.array(average_timedelta))).astype('timedelta64[m]').astype(int))
#dt = 5 #min

deltax = [data['x'][i-1]-data['x'][i] for i in range(1, len(data['x']))]
dxy = np.abs(np.mean(deltax).astype(int))/1000

print(dxy,dt) 

0.5 5


In [43]:
#This method uses a tobac utility to find spacing in meters/seconds
dxy,dt = tobac.utils.general.get_spacings(maxrefl.to_iris(), grid_spacing=None, time_spacing=None)
print(dxy)
print(dt)

500.0
181


In [44]:
maxrefl_iris=maxrefl.to_iris()
# Feature detection based on based on surface precipitation field and a range of thresholds
print('starting feature detection based on multiple thresholds')
Features_iris=tobac.feature_detection_multithreshold(maxrefl_iris,dxy,**parameters_features)
Features=Features_iris.to_xarray()
print('feature detection done')
Features.to_netcdf(os.path.join(savedir,'Features.nc'))
print('features saved')

starting feature detection based on multiple thresholds
feature detection done
features saved


In [45]:
# Dictionary containing keyword arguments for segmentation step:
parameters_segmentation={}
parameters_segmentation['method']='watershed'
parameters_segmentation['threshold']= track_threshold #15  # mm/h mixing ratio
#parameters_segmentation['ISO_dilate']=10 #this is the size 
#parameters_segmentation['features']
#parameters_segmentation['field']
#parameters_segmentation['dxy']
#parameters_segmentation['target']
#parameters_segmentation['level']
#parameters_segmentation['max_distance']
#Maximum distance from a marker allowed to be classified as
        #belonging to that cell. Default is None.
#parameters_segmentation['vertical_coord']

In [46]:
Features_df=Features.to_dataframe()

# Perform Segmentation and save resulting mask to NetCDF file:
print('Starting segmentation based on reflectivity')
Mask_iris,Features_Precip =tobac.segmentation.segmentation(Features_df,maxrefl_iris,dxy,**parameters_segmentation)
#Mask,Features_Precip=tobac.themes.tobac_v1.segmentation(Features,maxrefl,dxy,**parameters_segmentation)
Features_Precip=Features_Precip.to_xarray()
Mask=xr.DataArray.from_iris(Mask_iris)
Mask = Mask.to_dataset()


#Mask,Features_Precip=segmentation(Features,maxrefl,dxy,**parameters_segmentation)
print('segmentation based on reflectivity performed, start saving results to files')
Mask.to_netcdf(os.path.join(savedir,'Mask_Segmentation_refl.nc'))                
Features_Precip.to_netcdf(os.path.join(savedir,'Features_Precip.nc'))
print('segmentation reflectivity performed and saved')

Starting segmentation based on reflectivity
segmentation based on reflectivity performed, start saving results to files
segmentation reflectivity performed and saved


In [47]:
# Mask=xarray.DataArray.from_iris(Mask_iris)

In [48]:
#Finds area of feature

areas = np.zeros([(len(Features['index'])+1)])
maxfeature_refl = np.zeros([(len(Features['index'])+1)])
#Mask = Mask.to_dataset()
frame_features = Features.groupby('frame')

for frame_i, features_i in frame_features: 
    mask_i = Mask['segmentation_mask'][frame_i,:,:].values
    subrefl = maxrefl[frame_i,:,:].values
    for i in np.unique(mask_i):
        feature_area_i = np.where(mask_i == i)
        areas[i] = len(feature_area_i[0])
        maxfeature_refl[i] = np.nanmax(subrefl[feature_area_i])


var = Features['feature'].copy(data = areas[1:])
var = var.rename("areas")
var_max = Features['feature'].copy(data = maxfeature_refl[1:])
var_max = var_max.rename("max_reflectivity")
Features_Precip = xr.merge([Features_Precip,var], compat = 'override')
Features = xr.merge([Features,var], compat = 'override')
Features = xr.merge([Features,var_max], compat = 'override')
Features_Precip.to_netcdf(os.path.join(savedir, 'Features_Precip.nc'))
Features.to_netcdf(os.path.join(savedir,'Features.nc'))
Mask = Mask.to_array()
Features_df=Features.to_dataframe()
print('features saved')

features saved


In [49]:
# Dictionary containing keyword arguments for the linking step:
parameters_linking={}
parameters_linking['stubs'] = 3 #5
parameters_linking['method_linking']='predict'
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['extrapolate']=0#2 #was 1 - the number of frames to extrapolate to
parameters_linking['order']=2 #Order of polynomial for extrapolating
parameters_linking['subnetwork_size']=100 
parameters_linking['memory']= 3#4
#parameters_linking['time_cell_min']=1
parameters_linking['v_max']= 20 #if using m/s dxy/dt
parameters_linking['d_min']= None #5 

In [108]:
# Perform trajectory linking using trackpy and save the resulting DataFrame:

#Track=tobac.themes.tobac_v1.linking_trackpy(Features,Mask,dt=dt,dxy=dxy,**parameters_linking)
Features_df=Features.to_dataframe()
Track=tobac.linking_trackpy(Features_df,Mask_iris,dt=dt,dxy=dxy,**parameters_linking)
print(type(Track))
Track = Track.to_xarray()
Track.to_netcdf(os.path.join(savedir,'Track.nc'))

Frame 84: 69 trajectories present.
<class 'pandas.core.frame.DataFrame'>


In [109]:
# savedir = "/Users/kelcy/PYTHON/TRACER-PAWS-NEXRAD-LMA/scripts/Tracking/tobac_Save_20220617"

Track = xr.open_dataset(savedir+"/Track.nc")
# # Track = Track.to_dataframe()
# Features = xr.open_dataset(savedir+"/Features.nc")
refl_mask = xr.open_dataset(savedir+"/Mask_Segmentation_refl.nc")
# Mask = xarray.open_dataarray(savedir+"/Mask_Segmentation_refl.nc")
# Mask_iris = Mask.to_iris()

# refl_features = xarray.open_dataset(savedir+"/Features_Precip.nc")
# both_ds = xr.open_dataset(savedir+'/Track_features_merges.nc')

In [57]:
def compress_all(nc_grids, min_dims=2, comp_level=4):
    """
    The purpose of this subroutine is to compress the netcdf variables as they are saved.
    This does not change the data, but sets netcdf encoding parameters.
    We allocate a minimum number of dimensions as variables with dimensions
    under the minimum value do not benefit from tangibly from this encoding.

    Parameters
    ----------
    nc_grids : xarray.core.dataset.Dataset
        Xarray dataset that is intended to be exported as netcdf

    min_dims : integer
        The minimum number of dimesnions, in integer value, a variable must have in order
        set the netcdf compression encoding.
    comp_level : integer
        The level of compression. Default values is 4.

    Returns
    -------
    nc_grids : xarray.core.dataset.Dataset
        Xarray dataset with netcdf compression encoding for variables with two (2) or more dimensions

    """

    for var in nc_grids:
        if len(nc_grids[var].dims) >= min_dims:
            # print("Compressing ", var)
            nc_grids[var].encoding["zlib"] = True
            nc_grids[var].encoding["complevel"] = comp_level
            nc_grids[var].encoding["contiguous"] = False
    return nc_grids

In [112]:
# d, track_id = merge_split(Track,distance = 20000.)
import time
tic = time.perf_counter()


d = merge_split_MEST(Track.to_dataframe(),dxy, distance=25000.0)
toc = time.perf_counter()
print(toc - tic)
# # d = d.to_dataframe()
Track = xr.open_dataset(savedir+"/Track.nc")
ds = tobac.utils.standardize_track_dataset(Track, refl_mask)
both_ds = xr.merge([ds, d],compat ='override')

both_ds = compress_all(both_ds)
both_ds.to_netcdf(os.path.join(savedir,'Track_features_merges.nc'))

2.763929917000496


In [ ]:
#Below is code to find the number of neighbors, currently set to number of neighbors in r=5km
import numpy as np
from scipy.spatial import KDTree
hdim1 = both_ds['feature_hdim1_coordinate'].values*0.5
hdim2 = both_ds['feature_hdim2_coordinate'].values*0.5
pts = np.vstack((hdim2, hdim1)).T
tree = KDTree(pts)
#note hdim is in km on the grid
num_obj = np.zeros(len(both_ds["feature"].values))
for i,ind in enumerate(both_ds["feature"].values):
    num_obj[i]=len(tree.query_ball_point(pts[i],r=5))
num_obj = num_obj.astype(int)
both_ds = both_ds.assign(feature_nearby_count=(['feature'], num_obj))
both_ds.to_netcdf(os.path.join(savedir,'Track_features_merges.nc'))

In [ ]:
#
frame = 60
isolated_min = 0.5
show_tracks = True
ref_levels = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75]

fig, ax = plt.subplots(figsize=(10,10))

refl = maxrefl[frame,:,:] 
fig.suptitle(str(data['time'][frame].data)[:-10])
y_mesh,x_mesh = np.meshgrid(data['x'],data['y'])
    
refplt = ax.contourf(y_mesh,x_mesh, refl, extend = 'max',levels = ref_levels,cmap='pyart_LangRainbow12',origin = 'lower', vmin=-24, vmax=72)#,extent = [0,-10000,-20000,-10000])

fig.colorbar(refplt,fraction=0.046, pad=0.04)
i = np.where(refl_mask['segmentation_mask'][frame,:,:] > 0)
    

y, x = y_mesh[i[0],i[1]],x_mesh[i[0],i[1]]
# y,x, = nc_grid['point_longitude'][0,i[0],i[1]],nc_grid['point_latitude'][0,i[0],i[1]]
imcell2 = ax.scatter(y,x,s = 0.1,c = 'gray', marker = '.',alpha = 0.25)
    

#version for plotting cells
# for i in both_ds['cell']:
#     if i.values < 0:
#         continue
#     #print(i)
#     if math.isfinite(i):
#         track_i = np.where(both_ds['feature_parent_cell_id'] == i)
#         if (np.nanmax(both_ds['feature_time_index'][track_i]) >= frame) and (np.nanmin(both_ds['feature_time_index'][track_i]) <= frame):
# #         ]
# #         if np.any
# #         if np.any(Track['frame'][track_i] == frame):
#             ax.plot(both_ds['feature_projection_x_coordinate'][track_i], both_ds['feature_projection_y_coordinate'][track_i], '-.',color='r')
#             #ax.plot(Track['x'][track_i], Track['y'][track_i], '-.',color='r')
#             ax.text(both_ds['feature_projection_x_coordinate'][track_i][-1],both_ds['feature_projection_y_coordinate'][track_i][-1], f'{int(i)}', fontsize = 'small',rotation = 'vertical')
#         else:
#             continue
 
#version for plotting tracks
for i in both_ds['track']:
#     if i < val :
#         continue
#     if i > val:
#         continue
    track_i = np.where(both_ds['cell_parent_track_id'] == i.values)
    for cell in both_ds['cell'][track_i]:
        if cell < 0:
            continue

        feature_id = np.where(both_ds['feature_parent_cell_id'] == cell)
        if (frame <= np.nanmax(both_ds['feature_time_index'][feature_id])) and (frame >= np.nanmin(both_ds['feature_time_index'][feature_id])):
            ax.plot(both_ds['feature_projection_x_coordinate'][feature_id], both_ds['feature_projection_y_coordinate'][feature_id], '-.',color='b',alpha = 0.5)
            ax.text(both_ds['feature_projection_x_coordinate'][feature_id][-1],both_ds['feature_projection_y_coordinate'][feature_id][-1], f'{int(i)}', fontsize = 'small',rotation = 'vertical')
    else:
        continue

     fig.savefig(plot_dir+'/'+'20260331_track_'+str(frame)+'.png')




In [128]:
import numpy as np
import warnings
import xarray as xr

from scipy import ndimage
from datetime import datetime

try:
    import pyproj
    _PYPROJ_AVAILABLE = True
except ImportError:
    _PYPROJ_AVAILABLE = False

def cartesian_to_geographic_aeqd(x, y, lon_0, lat_0, R=6370997.):
    """
    Azimuthal equidistant Cartesian to geographic coordinate transform.

    Transform a set of Cartesian/Cartographic coordinates (x, y) to
    geographic coordinate system (lat, lon) using a azimuthal equidistant
    map projection [1]_.

    .. math::

        lat = \\arcsin(\\cos(c) * \\sin(lat_0) +
                       (y * \\sin(c) * \\cos(lat_0) / \\rho))

        lon = lon_0 + \\arctan2(
            x * \\sin(c),
            \\rho * \\cos(lat_0) * \\cos(c) - y * \\sin(lat_0) * \\sin(c))

        \\rho = \\sqrt(x^2 + y^2)

        c = \\rho / R

    Where x, y are the Cartesian position from the center of projection;
    lat, lon the corresponding latitude and longitude; lat_0, lon_0 are the
    latitude and longitude of the center of the projection; R is the radius of
    the earth (defaults to ~6371 km). lon is adjusted to be between -180 and
    180.

    Parameters
    ----------
    x, y : array-like
        Cartesian coordinates in the same units as R, typically meters.
    lon_0, lat_0 : float
        Longitude and latitude, in degrees, of the center of the projection.
    R : float, optional
        Earth radius in the same units as x and y. The default value is in
        units of meters.

    Returns
    -------
    lon, lat : array
        Longitude and latitude of Cartesian coordinates in degrees.

    References
    ----------
    .. [1] Snyder, J. P. Map Projections--A Working Manual. U. S. Geological
        Survey Professional Paper 1395, 1987, pp. 191-202.

    """
    x = np.atleast_1d(np.asarray(x))
    y = np.atleast_1d(np.asarray(y))

    lat_0_rad = np.deg2rad(lat_0)
    lon_0_rad = np.deg2rad(lon_0)

    rho = np.sqrt(x*x + y*y)
    c = rho / R

    with warnings.catch_warnings():
        # division by zero may occur here but is properly addressed below so
        # the warnings can be ignored
        warnings.simplefilter("ignore", RuntimeWarning)
        lat_rad = np.arcsin(np.cos(c) * np.sin(lat_0_rad) +
                            y * np.sin(c) * np.cos(lat_0_rad) / rho)
    lat_deg = np.rad2deg(lat_rad)
    # fix cases where the distance from the center of the projection is zero
    lat_deg[rho == 0] = lat_0

    x1 = x * np.sin(c)
    x2 = rho*np.cos(lat_0_rad)*np.cos(c) - y*np.sin(lat_0_rad)*np.sin(c)
    lon_rad = lon_0_rad + np.arctan2(x1, x2)
    lon_deg = np.rad2deg(lon_rad)
    # Longitudes should be from -180 to 180 degrees
    lon_deg[lon_deg > 180] -= 360.
    lon_deg[lon_deg < -180] += 360.

    return lon_deg, lat_deg

def cartesian_to_geographic(grid_ds):
    """
    Cartesian to Geographic coordinate transform.

    Transform a set of Cartesian/Cartographic coordinates (x, y) to a
    geographic coordinate system (lat, lon) using pyproj or a build in
    Azimuthal equidistant projection.

    Parameters
    ----------
    grid_ds: xarray DataSet
        Cartesian coordinates in meters unless R is defined in different units
        in the projparams parameter.

    Returns
    -------
    lon, lat : array
        Longitude and latitude of the Cartesian coordinates in degrees.

    """
    projparams = grid_ds.ProjectionCoordinateSystem
    x = grid_ds.x.values
    y = grid_ds.y.values
    z = grid_ds.z.values
    z, y, x = np.meshgrid(z, y, x, indexing='ij')
    if projparams.attrs['grid_mapping_name'] == 'azimuthal_equidistant':
        # Use Py-ART's Azimuthal equidistance projection
        lat_0 = projparams.attrs['latitude_of_projection_origin']
        lon_0 = projparams.attrs['longitude_of_projection_origin']
        if 'semi_major_axis' in projparams:
            R = projparams.attrs['semi_major_axis']
            lon, lat = cartesian_to_geographic_aeqd(x, y, lon_0, lat_0, R)
        else:
            lon, lat = cartesian_to_geographic_aeqd(x, y, lon_0, lat_0)
    else:
        # Use pyproj for the projection
        # check that pyproj is available
        if not _PYPROJ_AVAILABLE:
            raise MissingOptionalDependency(
                "PyProj is required to use cartesian_to_geographic "
                "with a projection other than pyart_aeqd but it is not "
                "installed")
        proj = pyproj.Proj(projparams)
        lon, lat = proj(x, y, inverse=True)
    return lon, lat


def add_lat_lon_grid(grid_ds):
    lon, lat = cartesian_to_geographic(grid_ds)
    grid_ds["point_latitude"] = xr.DataArray(lat, dims=["z", "y", "x"])
    grid_ds["point_latitude"].attrs["long_name"] = "Latitude"
    grid_ds["point_latitude"].attrs["units"] = "degrees"
    grid_ds["point_longitude"] = xr.DataArray(lon, dims=["z", "y", "x"])
    grid_ds["point_longitude"].attrs["long_name"] = "Latitude"
    grid_ds["point_longitude"].attrs["units"] = "degrees"
    return grid_ds

def parse_grid_datetime(my_ds):
    year = my_ds['time'].dt.year
    month = my_ds['time'].dt.month
    day = my_ds['time'].dt.day
    hour = my_ds['time'].dt.hour
    minute = my_ds['time'].dt.minute
    second = my_ds['time'].dt.second
    return datetime(year=year, month=month, day=day,
                    hour=hour, minute=minute, second=second)


""" X-Array based TINT I/O module. """

import xarray as xr
import random
import numpy as np
import pyproj

#from .grid_utils import add_lat_lon_grid
from datetime import datetime

def load_cfradial_grids(file_list):
    ds = xr.open_mfdataset(file_list)
    # Check for CF/Radial conventions
    if not ds.attrs["Conventions"] == 'CF/Radial instrument_parameters':
        ds.close()
        raise IOError("TINT module is only compatible with CF/Radial files!")
    ds = add_lat_lon_grid(ds)


    return ds

nc_grid = load_cfradial_grids(path+"*grid.nc")

In [129]:
nc_grid

<xarray.Dataset>
Dimensions:                     (time: 85, x: 1001, y: 1001, z: 31, nradar: 1)
Coordinates:
  * time                        (time) datetime64[ns] 2023-06-21T20:00:29.459...
  * x                           (x) float64 -2.5e+05 -2.495e+05 ... 2.5e+05
  * y                           (y) float64 -2.5e+05 -2.495e+05 ... 2.5e+05
  * z                           (z) float64 0.0 500.0 1e+03 ... 1.45e+04 1.5e+04
Dimensions without coordinates: nradar
Data variables: (12/23)
    origin_latitude             (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    origin_longitude            (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    origin_altitude             (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    projection                  (time) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    ProjectionCoordinateSystem  (time) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    radar_latitude              (time, nradar) float64 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ...                          ...
    MU                          (time, z, y, x) float32 dask.array<chunksize=(1, 31, 1001, 1001), meta=np.ndarray>
    MW                          (time, z, y, x) float32 dask.array<chunksize=(1, 31, 1001, 1001), meta=np.ndarray>
    MI                          (time, z, y, x) float32 dask.array<chunksize=(1, 31, 1001, 1001), meta=np.ndarray>
    ROI                         (time, z, y, x) float32 dask.array<chunksize=(1, 31, 1001, 1001), meta=np.ndarray>
    point_latitude              (z, y, x) float64 28.45 28.45 ... 32.94 32.94
    point_longitude             (z, y, x) float64 -99.94 -99.93 ... -94.71 -94.7
Attributes: (12/16)
    Conventions:         CF/Radial instrument_parameters
    version:             1.3
    title:               
    institution:         
    references:          
    source:              
    ...                  ...
    field_names:         reflectivity, spectrum_width, velocity, differential...
    history:             
    volume_number:       0
    platform_type:       fixed
    instrument_type:     radar
    primary_axis:        axis_z

In [ ]:
# longitude = []
# for i in range(len(Track['time'])):
#     #longitude.append(np.mean(nc_grid['point_longitude'].sel({'x':Track['x'].data[i]},method='nearest').data))       
#     longitude.append(np.mean(nc_grid['point_longitude'].sel({'x':Track['projection_x_coordinate'].data[i]},method='nearest').data))       


    
# latitude = []
# for i in range(len(Track['time'])):
#     #latitude.append(np.mean(nc_grid['point_latitude'].sel({'y':Track['y'].data[i]},method='nearest').data))     
#     latitude.append(np.mean(nc_grid['point_latitude'].sel({'y':Track['projection_y_coordinate'].data[i]},method='nearest').data))     


# latitude = np.array(latitude)
# longitude = np.array(longitude)
# #Track['latitude'] = latitude
# #Track['longitude'] = longitude

# nc_lons = nc_grid['point_longitude'][0,:,:].data
# nc_lats = nc_grid['point_latitude'][0,:,:].data

In [ ]:
#USING BOTH_DS XARRAY COMBINED DATA SET

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.mpl.geoaxes import GeoAxes
from mpl_toolkits.axes_grid1 import AxesGrid

def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end
    

def plot(t_index,xrdata,max_refl,ncgrid,dbz,ind = None):
    # Get the data
    hsv_ctr_lat, hsv_ctr_lon = 29.4719, -95.0792

    refl = max_refl[t_index,:,:]

    t_step = str(ncgrid['time'][t_index].values)
    nclons = ncgrid['point_longitude'][0,:,:].data
    nclats = ncgrid['point_latitude'][0,:,:].data
    
    fname = "/Users/kelcy/Downloads/10m_cultural/10m_cultural/ne_10m_admin_1_states_provinces_lines.shp"
    # Plot
    #fig.clear()
    latlon_proj = ccrs.PlateCarree()
    cs_attrs = ncgrid['ProjectionCoordinateSystem'][0].attrs
    if cs_attrs['grid_mapping_name'] == 'azimuthal_equidistant':
        grid_proj = ccrs.AzimuthalEquidistant(central_latitude=cs_attrs['latitude_of_projection_origin'],
                central_longitude=cs_attrs['longitude_of_projection_origin'],
                false_easting=cs_attrs['false_easting'],
                false_northing=cs_attrs['false_northing'],)
    projection=grid_proj
    axes_class = (GeoAxes,
                  dict(map_projection=projection))
    axs = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(1, 1),
                axes_pad=0.4,
                cbar_location='right',
                cbar_mode='each',
                cbar_pad=0.4,
                cbar_size='3%',
                label_mode='')  # note the empty label_mode
    for ax in axs:
        ax.coastlines()
        shape_feature = ShapelyFeature(Reader(fname).geometries(),
                                ccrs.PlateCarree(), edgecolor='black')
        ax.add_feature(shape_feature, facecolor='none')
        ax.add_feature(cartopy.feature.STATES, edgecolor = 'black')
        ax.set_extent((hsv_ctr_lon-2.5, hsv_ctr_lon+2.5,
                    hsv_ctr_lat-3.0, hsv_ctr_lat+2.5))    
#         ax.set_extent((hsv_ctr_lon-1.0, hsv_ctr_lon+1.5,
#                     hsv_ctr_lat-1.5, hsv_ctr_lat+1.0))
#         ax.set_extent((hsv_ctr_lon-1.0, hsv_ctr_lon+0.5,
#                     hsv_ctr_lat-0.5, hsv_ctr_lat+0.75))
        
        gl = ax.gridlines(draw_labels=True)
        gl.top_labels = False
        gl.right_labels = False
 
    # Gridded background
    grid_extent = (ncgrid.x.min(), ncgrid.x.max(),
                   ncgrid.y.min(), ncgrid.y.max())
    
    #fig.suptitle((t_step[0:19] + ' 40 dbz, long tracks, ISO_THRESH = 12'), fontsize = 12,y=0.76)


    # Cell ID
    im = axs[0].imshow(refl,origin = 'lower', vmin=-25, vmax=85, cmap='pyart_LangRainbow12',extent = grid_extent, transform = grid_proj)
    axs[0].set_title((t_step[0:19]+' ' + str(dbz)+' dbz, Tobac'))
    axs.cbar_axes[0].colorbar(im)
    

# #     track_cell = track.groupby('cell')
# #     times = np.unique(track['time'])
# #     track_time = track.groupby('time')

    i = np.where(xrdata['segmentation_mask'][t_index,:,:] > 0)
    y1, x1 = ncgrid['point_longitude'].data[0,i[0],i[1]],ncgrid['point_latitude'].data[0,i[0],i[1]]

    axs[0].scatter(y1,x1, s = 1,c = 'gray', marker = '.',alpha = 0.3, transform = latlon_proj)

    
#     for i in xrdata['cell']:
#         if i < 0:
#             continue

#     #print(i)
#         if math.isfinite(i):
#             track_i = np.where(xrdata['feature_parent_cell_id'] == i)
#             if (np.nanmax(xrdata['feature_time_index'][track_i]) >= t_index) and (np.nanmin(xrdata['feature_time_index'][track_i]) <= t_index):
#                 axs[0].plot(ncgrid['point_longitude'].data[0,np.round(xrdata['feature_hdim1_coordinate'].data[track_i]).astype(int),np.round(xrdata['feature_hdim2_coordinate'].data[track_i]).astype(int)],
#                         ncgrid['point_latitude'].data[0,np.round(xrdata['feature_hdim1_coordinate'].data[track_i]).astype(int),np.round(xrdata['feature_hdim2_coordinate'].data[track_i]).astype(int)],
#                         '-.',color = 'r',markersize = 1,transform = latlon_proj)
#                 axs[0].text(ncgrid['point_longitude'].data[0,np.round(both_ds['feature_hdim1_coordinate'].data[track_i][-1]).astype(int),np.round(both_ds['feature_hdim2_coordinate'].data[track_i][-1]).astype(int)],
#                         ncgrid['point_latitude'].data[0,np.round(both_ds['feature_hdim1_coordinate'].data[track_i][-1]).astype(int),np.round(both_ds['feature_hdim2_coordinate'].data[track_i][-1]).astype(int)],
#                         f'{int(i)}', fontsize = 'small',rotation = 'vertical',transform = latlon_proj)
#         else:
#             continue


    for i in xrdata['track']:
        track_i = np.where(xrdata['cell_parent_track_id'] == i.values)
        for cell in xrdata['cell'][track_i]:
            if cell < 0:
                continue

            feature_id = np.where(xrdata['feature_parent_cell_id'] == cell)
            if (np.nanmax(xrdata['feature_time_index'][feature_id]) >= t_index) and (np.nanmin(xrdata['feature_time_index'][feature_id]) <= t_index):
                axs[0].plot(ncgrid['point_longitude'].data[0,np.round(xrdata['feature_hdim1_coordinate'].data[feature_id]).astype(int),np.round(xrdata['feature_hdim2_coordinate'].data[feature_id]).astype(int)],
                        ncgrid['point_latitude'].data[0,np.round(xrdata['feature_hdim1_coordinate'].data[feature_id]).astype(int),np.round(xrdata['feature_hdim2_coordinate'].data[feature_id]).astype(int)],
                        '-.',color = 'b',markersize = 1,transform = latlon_proj)
            
                axs[0].text(ncgrid['point_longitude'].data[0,np.round(both_ds['feature_hdim1_coordinate'].data[feature_id][-1]).astype(int),np.round(both_ds['feature_hdim2_coordinate'].data[feature_id][-1]).astype(int)],
                        ncgrid['point_latitude'].data[0,np.round(both_ds['feature_hdim1_coordinate'].data[feature_id][-1]).astype(int),np.round(both_ds['feature_hdim2_coordinate'].data[feature_id][-1]).astype(int)],
                        f'{int(i)}', fontsize = 'small',rotation = 'vertical',transform = latlon_proj)
            else:
                continue
    
    
    

    

In [ ]:
# for i in range(len(nc_grid.time)):
i = 90
time_index = i

fig = plt.figure(figsize=(9,9))
fig.set_canvas(plt.gcf().canvas)
plot(time_index,both_ds,maxrefl,nc_grid,30)
